In [1]:
from keras import backend as K 
import math 
import tensorflow as tf
@tf.function
def custom_loss2(y_true, y_pred):
    alpha = 0.2
    difference = y_pred - y_true
    squared_difference = tf.square(y_pred - y_true)
    
    # Calcola la loss per ciascun elemento
    loss = tf.where(difference < 0, 2 * alpha * squared_difference, 2 * (alpha + (1 - 2 * alpha)) * squared_difference)
    
    # Calcola la media delle loss
    return tf.reduce_mean(loss)

@tf.function
def custom_score(y_true, y_pred):
    d_i = y_pred - y_true
    #esponente = tf.where(d_i < 0, 1.0 / (d_i / 13.0), d_i / 10.0)
    sum = tf.reduce_sum(tf.where(d_i < 0, tf.exp(-d_i/13)-1, tf.exp(d_i/10)-1)) #prova
    #sum = tf.reduce_sum(tf.exp(esponente) - 1.0)
    return sum




2023-10-08 22:33:15.272299: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-08 22:33:17.091187: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/aliquodfahriam/miniconda3/envs/tf/lib/:/lib
2023-10-08 22:33:17.091435: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/aliquodfahriam/miniconda3/envs/tf/lib/:/lib
2023-10-08 22:33:17.091444: 

In [3]:
small_lstm = tf.keras.models.load_model('./models/LSTMsmall/FD0001_scoreFixed/', custom_objects={'custom_loss2':custom_loss2, 'custom_score':custom_score})
large_lstm = tf.keras.models.load_model('./models/LSTMlarge/FD0001/', custom_objects={'custom_loss2':custom_loss2, 'custom_score':custom_score})

converter = tf.lite.TFLiteConverter.from_keras_model(small_lstm)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
lstm_small_tflite = converter.convert()
with open("small_lstm.tflite", "wb") as f:
    f.write(lstm_small_tflite)

converter2 = tf.lite.TFLiteConverter.from_keras_model(large_lstm)
converter2.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
lstm_large = converter2.convert()
with open("large_lstm.tflite", "wb") as f:
    f.write(lstm_small_tflite)

INFO:tensorflow:Assets written to: /tmp/tmpwdrf5v7j/assets


INFO:tensorflow:Assets written to: /tmp/tmpwdrf5v7j/assets
2023-10-08 22:34:33.754199: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-10-08 22:34:33.754225: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-10-08 22:34:34.685306: W tensorflow/compiler/mlir/lite/flatbuffer_export.cc:2046] TFLite interpreter needs to link Flex delegate in order to run the model since it contains the following Select TFop(s):
Flex ops: FlexTensorListReserve, FlexTensorListSetItem, FlexTensorListStack
Details:
	tf.TensorListReserve(tensor<2xi32>, tensor<i32>) -> (tensor<!tf_type.variant<tensor<?x30xf32>>>) : {device = ""}
	tf.TensorListReserve(tensor<2xi32>, tensor<i32>) -> (tensor<!tf_type.variant<tensor<?x60xf32>>>) : {device = ""}
	tf.TensorListSetItem(tensor<!tf_type.variant<tensor<?x30xf32>>>, tensor<i32>, tensor<?x30xf32>) -> (tensor<!tf_type.variant<tensor<?x30xf32>>>) : {device = ""}
	

INFO:tensorflow:Assets written to: /tmp/tmphk0z0hmn/assets


INFO:tensorflow:Assets written to: /tmp/tmphk0z0hmn/assets
2023-10-08 22:35:01.234598: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-10-08 22:35:01.234635: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-10-08 22:35:02.927100: W tensorflow/compiler/mlir/lite/flatbuffer_export.cc:2046] TFLite interpreter needs to link Flex delegate in order to run the model since it contains the following Select TFop(s):
Flex ops: FlexTensorListReserve, FlexTensorListSetItem, FlexTensorListStack
Details:
	tf.TensorListReserve(tensor<2xi32>, tensor<i32>) -> (tensor<!tf_type.variant<tensor<?x128xf32>>>) : {device = ""}
	tf.TensorListReserve(tensor<2xi32>, tensor<i32>) -> (tensor<!tf_type.variant<tensor<?x16xf32>>>) : {device = ""}
	tf.TensorListReserve(tensor<2xi32>, tensor<i32>) -> (tensor<!tf_type.variant<tensor<?x32xf32>>>) : {device = ""}
	tf.TensorListReserve(tensor<2xi32>, tensor<i32